In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2026-01-16 17:15:03.286 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/max/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 2381.55it/s]


In [3]:
len(train), len(valid)

(81, 20)

In [4]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 29, 3]),
 tensor([13, 15,  7,  2,  9,  9, 18,  1,  8,  5,  0,  4, 10, 13,  9, 11,  3, 12,
          8, 19,  4,  3, 16,  9,  8,  6,  4, 10,  6, 13,  3,  7]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [5]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [6]:
model = rnn_models.BaseRNN(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    horizon=20,
)

Test the model. What is the output shape you need? Remember, we are doing classification!

In [7]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [8]:
accuracy(y, yhat)

0.03125

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [9]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [10]:
yhat[0]

tensor([-0.0004,  0.0448, -0.1127,  0.1066,  0.0844,  0.0416,  0.0629, -0.0734,
         0.1544,  0.0686,  0.1114, -0.0614,  0.0792,  0.0144,  0.0656, -0.0494,
        -0.0075,  0.0694,  0.0657,  0.1740], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(2.9989, grad_fn=<NllLossBackward0>)

In [12]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [13]:
settings = TrainerSettings(
    epochs=12, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 12
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [14]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [15]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("model", "GRU")
    config = ModelConfig(
        input_size=3,
        hidden_size=128 + 32,
        num_layers=3,
        output_size=20,
        dropout=0.2,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

mlflow.end_run()

2026-01-16 16:40:24.991 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20260116-164024
2026-01-16 16:40:24.993 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:03<00:00, 25.23it/s]
2026-01-16 16:40:28.870 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.4516 test 2.2284 metric ['0.2234']
100%|██████████| 81/81 [00:03<00:00, 25.77it/s]
2026-01-16 16:40:32.795 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.8482 test 1.4821 metric ['0.4109']
100%|██████████| 81/81 [00:02<00:00, 28.18it/s]
2026-01-16 16:40:36.453 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.1733 test 0.9682 metric ['0.6141']
100%|██████████| 81/81 [00:03<00:00, 24.66it/s]
2026-01-16 16:40:40.432 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.6827 test 0.5374 metric ['0.8219']
100%|██████████| 81/81 [00:03<00:00, 25.88it/s]
2026-01-16 16:40:44.28

🏃 View run melodic-snake-593 at: http://localhost:5000/#/experiments/827571558083763418/runs/a894e82e9a1b49069e5aaa5351d88801
🧪 View experiment at: http://localhost:5000/#/experiments/827571558083763418


Try to update the code above by changing the hyperparameters.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [33]:
settings_lstm = TrainerSettings(
    epochs=16, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)

class LSTMmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.LSTM(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

with mlflow.start_run():
    mlflow.set_tag("model", "LSTM")
    config = ModelConfig(
        input_size=3,
        output_size=20,
        hidden_size=96,
        num_layers=2,
        dropout=0.2,
    )

    model = LSTMmodel(config)

    trainer = Trainer(
        model=model,
        settings=settings_lstm,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings_lstm.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

mlflow.end_run()

2026-01-16 17:36:59.412 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20260116-173659
2026-01-16 17:36:59.413 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:00<00:00, 87.56it/s]
2026-01-16 17:37:00.931 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.7076 test 2.3858 metric ['0.1547']
100%|██████████| 81/81 [00:00<00:00, 111.35it/s]
2026-01-16 17:37:02.229 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1784 test 2.0297 metric ['0.2469']
100%|██████████| 81/81 [00:00<00:00, 131.00it/s]
2026-01-16 17:37:03.418 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.8510 test 1.7422 metric ['0.3047']
100%|██████████| 81/81 [00:00<00:00, 138.29it/s]
2026-01-16 17:37:04.573 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.4574 test 1.3617 metric ['0.4047']
100%|██████████| 81/81 [00:00<00:00, 147.85it/s]
2026-01-16 17:37:0

🏃 View run angry-wasp-119 at: http://localhost:5000/#/experiments/827571558083763418/runs/caa173034f1240e19cbc476130ea964c
🧪 View experiment at: http://localhost:5000/#/experiments/827571558083763418


In [17]:
# trainer.loop() # if you want to pick up training, loop will continue from the last epoch

In [ ]:
# mlflow.end_run()